In [ ]:
from PIL import Image
import numpy as np
import cv2
import numpy as np
import os
from xml.etree.ElementTree import XML, fromstring, tostring, ElementTree
import winsound

In [ ]:
# This function performs affine transformation. Inputs are: a string variable which is the 
# name (with path) of the image to transform, three float variables, x and y displacement and 
# the scale, which will be equal for both dimensions of the picture; the scale only works for
# values between 0 and 2, not included.


def myaffine(immagine,disp_x,disp_y,scale):
    img = cv2.imread(immagine)
    M = np.float32([[1,0,disp_x],[0,1,disp_y]]) # traslation matrix
    dst = cv2.warpAffine(img,M,(img.shape[1],img.shape[0]))  #dst is the figure displaced
    res = cv2.resize(dst,None,fx=scale, fy=scale, interpolation = cv2.INTER_LINEAR) #res is the figure resized
        
    return res

In [ ]:
#this function variates the gamma of the picture given in input as a string

def adjust_gamma(image, gamma=1.0):

   invGamma = 1.0 / gamma
   table = np.array([((i / 255.0) ** invGamma) * 255
      for i in np.arange(0, 256)]).astype("uint8")

   return cv2.LUT(image, table)

In [ ]:
# This function gets a string as input, which is the path and name of the image that will be
# transformed as follows: random translation, rescaling, blur, paste of a disturbing image.
# It also takes track of the position of the rectangle in which the mode (or the interesting
# part of your picture) is inscribed; the output is an open cv2 transformed image and two 
# arrays with the rectangle coordinates.

def myrandtransform(immagine):
    disp_x, disp_y=np.random.uniform(-0.45,0.45,2) #chooses a random x and y percentage shift 
    scala=np.random.uniform(0.33,2) #chooses a random scale
    img = cv2.imread(immagine) #read the image
    xshape,yshape,colori=img.shape #gets image's shape
    shift=(int(xshape*disp_x),int(yshape*disp_y)) #transforms percentage shifts in displacements on the image
    
    aff=myaffine(immagine,shift[0],shift[1],scala) #operates shifts and rescaling
    affx,affy,afcolor=aff.shape
    
    fiore=myflower(aff) #gets a random flower picture
    x_corner_flow=np.random.randint(0,aff.shape[0]-fiore.shape[0]) #chooses a random position toplace the flower on the shifted and rescaled picture, for x...
    y_corner_flow=np.random.randint(0,aff.shape[1]-fiore.shape[1]) #...and y dimension
    aff[x_corner_flow:x_corner_flow+fiore.shape[0],y_corner_flow:y_corner_flow+fiore.shape[1]]=fiore #pastes the flower on the selected point
    
    adj = adjust_gamma(aff, gamma=np.random.uniform(1,2)) #changes gamma value
    
    do_blur=np.random.rand() #we choose wether to blur the image
    if do_blur<0.5:
        how_blur=np.random.randint(1,5)  #and how to blur it
        blur = cv2.blur(adj,(how_blur,how_blur))
    else: blur=adj
        
# Determination of the rectangle; a rectangle is identified by the x and y coordinates on 
# the picture of two corners: top left and bottom right. Note that coordinates in pictures 
# are defined as follows: the top left corner has coordinates (0,0), the bottom right has 
# coordinates (x,y)=(height,width)=(img.shape[0],img.shape[1])=
# (number of columns, number of rows)
    
    
    #We start from the identification of the rectangle in the original picture:
    A=(int(0.5*(xshape-ex)),int(0.5*(yshape-ey))) #top left (x,y)
    B=(A[0]+ex,A[1]+ey)                           #bottom right (x,y)
    
#calculation of the rectangle after all transformations. Note that it is necessary that
#the rectangle is in the picture in a limit of at least 5 pixels, in fact the machine-learning 
# code will compute the ratio between these rectangle coordinates and the image shape and this 
# ratio has to be smaller than 1.024127. For this reason we have the max and min evaluations
#and...
    
    x_top_left=max(5,int(scala*(A[0]+shift[0]))+int(0.25*scala*ex))
    y_top_left=max(5,int(scala*(A[1]+shift[1]))-int(0.25*scala*ey))
    a1=(x_top_left,y_top_left)
    x_bottom_right=min(affy-5,int(scala*(B[0]+shift[0]))+int(0.25*scala*ex))
    y_bottom_right=min(affx-5,int(scala*(B[1]+shift[1]))-int(0.25*scala*ey))
    b1=(x_bottom_right,y_bottom_right)
    
#... thesw control if
    
    if(float(a1[0]/affy)>1.024127):
        print(immagine+"\t a1[0] norm error")
    if(float(a1[1]/affx)>1.024127):
        print(immagine+"\t a1[1] norm error")
    if(float(b1[0]/affy)>1.024127):
        print(immagine+"\t b1[0] norm error")
    if(float(b1[1]/affx)>1.024127):
        print(immagine+"\t b1[1] norm error")
        
    #if you want to print the rectangle on your image uncomment the following 
    #(deprecated in the full generation cycle, suggested in a check of the program)
    
    #final = cv2.rectangle(blur, a1, b1, (0,255,0), 3)
    return (blur,a1,b1)

In [ ]:
# This function gets an open cv2 image in input, chooses randomly a picture of a flower
# which is an object similar to the modes we detect; you can choose different pictures 
# according to your purposes. These pictures are stored in a folder, whose content is 
# listed in mylist. The output is an open cv2 image.

def myflower(aff):
    n=np.random.randint(0,len(mylist)) #random choice
    f1 = cv2.imread(flower_path+mylist[n])
    fscala=min(1,np.random.uniform(0.05,0.35)*aff.shape[0]/f1.shape[0]) #we want the flower to be sufficiently small not to totally overlap with the mode
    fin_flow=myaffine(flower_path+mylist[n],0,0,fscala) #so we properly transform its picture
    return fin_flow

In [ ]:
# This function is useful if we want to generate pictures for training in wich there are two
# objects to recognize. As this function is used in a cycle, its input is teh integer step of 
#the cycle. The ouputs are the ones required to write the xml file.

def coppie(i): 
    lx,ly=600,600  #blank image dimensions
    blank = np.ones((lx,ly,3),np.uint8)
    
# We randomly choose two images in our original sampple
    
    s1=s_modi_studiati[np.random.randint(0,len(s_modi_studiati))]
    s2=s_modi_studiati[np.random.randint(0,len(s_modi_studiati))]
    im1=cv2.imread(path+'-'+s1+"_shape.jpg")
    im2=cv2.imread(path+'-'+s2+"_shape.jpg")

# Calculation of the rectangle in which the mode is inscribed in on of the original pictures
    
    xshape,yshape,colori=im1.shape
    A=(int(0.5*(xshape-ex)),int(0.5*(yshape-ey)))
    B=(A[0]+ex,A[1]+ey)

# We want to place our pictures on the blank one in such a way that their overlap is 
# small enough to guarantee a proper learning. Hence we define the variable overlap
# and set it to its maximum and a boolean variable flag, we start a cycle in which we 
# keep placing the images untill the overlap is sufficiently small
    
    overlap=1.0
    flag=1
    while(overlap>=0.15 and flag):
        rand_top_right_corner_1_x=np.random.randint(0,lx-ex)
        rand_top_right_corner_1_y=np.random.randint(0,ly-ey)
        rand_top_right_corner_2_x=np.random.randint(0,lx-ex)
        rand_top_right_corner_2_y=np.random.randint(0,ly-ey)
        A1=(rand_top_right_corner_1_x,rand_top_right_corner_1_y)
        A2=(rand_top_right_corner_2_x,rand_top_right_corner_2_y)
        B1=(A1[0]+ex,A1[1]+ey)
        B2=(A2[0]+ex,A2[1]+ey)
        flagx=abs(B2[0]-A1[0])   #x intersection
        flagy=abs(B2[1]-A1[1])   #y intersection
        if(flagx>2*ex or flagy>2*ey):
            flag=0              #it is a case in which the images do not overlap
        else: flag=1    #if they overlap, we calculate how big the overlap is
        ox=abs(B1[0]-A2[0])   #x side of the rectangle of the intersection
        oy=abs(B1[1]-A2[1])   #y side
        overlap=float(ox*oy)/(lx*ly)
    
    #outside the while cycle, the images are correctly placed and we can save the blank one
    if float(i/coppie_range) < 0.8:
        destin="C:/tensorflow1/models/research/object_detection/images/train/"
    else:
        destin="C:/tensorflow1/models/research/object_detection/images/test/"
    s="coppia"+str(i).zfill(3)+".jpg"
    blank[A1[0]:B1[0],A1[1]:B1[1]]=im1[A[0]:B[0],A[1]:B[1]]
    blank[A2[0]:B2[0],A2[1]:B2[1]]=im2[A[0]:B[0],A[1]:B[1]]
    cv2.imwrite(destin+s,blank)
    return (s,destin,['modo_meno_'+s1,'modo_meno_'+s2],A1,B1,A2,B2)
    

In [ ]:
# This function creates an ".xml" file which will be identical to the one created with a manual
# labelling software. Its inputs are:
#     flag: str, tells wether the image to be processed has one or two modes to be distiguished
#     filename: name of the file without path
#     name: name of the class in which the learner must place the image
#     path: str
#     other: int

    

def createxml(flag,filename,name,path,width,height,xmin,ymin,xmax,ymax):
    s1='<?xml version="1.0" ?>\n<annotation>\n\t\n\t\n\t<folder>test</folder>\n\t\n\t\n\t<filename>'
    s2='</filename>\n\t\n\t\n\t<path>'
    #attention, in the following JPG!
    s10='.JPG</path>\n\t\n\t\n\t<source>\n\t\t\n\t\t\n\t\t<database>Unknown</database>\n\t\t\n\t\n\t</source>\n\t\n\t\n\t<size>\n\t\t\n\t\t\n\t\t<width>'
    s3='</width>\n\t\t\n\t\t\n\t\t<height>'
    s4='</height>\n\t\t\n\t\t\n\t\t<depth>3</depth>\n\t\t\n\t\n\t</size>\n\t\n\t\n\t<segmented>0</segmented>\n\t\n\t\n\t<object>\n\t\t\n\t\t\n\t\t<name>'
    so5='</name>\n\t\t\n\t\t\n\t\t<pose>Unspecified</pose>\n\t\t\n\t\t\n\t\t<truncated>0</truncated>\n\t\t\n\t\t\n\t\t<difficult>0</difficult>\n\t\t\n\t\t\n\t\t<bndbox>\n\t\t\t\n\t\t\t\n\t\t\t<xmin>'
    so6='</xmin>\n\t\t\t\n\t\t\t\n\t\t\t<ymin>'
    so7='</ymin>\n\t\t\t\n\t\t\t\n\t\t\t<xmax>'
    so8='</xmax>\n\t\t\t\n\t\t\t\n\t\t\t<ymax>'
    so9='</ymax>\n\t\t\t\n\t\t\n\t\t</bndbox>\n\t\t\n\t\n\t</object>'
    soo='\n\t\n\t\n\t<object>\n\t\t\n\t\t\n\t\t<name>'
    fine='\n\t\n\n</annotation>\n'
    common_string=s1+filename+s2+path+filename+s10+width+s3+height+s4+name[0]
    repeated=so5+xmin[0]+so6+ymin[0]+so7+xmax[0]+so8+ymax[0]+so9
    if flag=='simple':
        my_xml_string=common_string+repeated+fine
    if flag=='coppie':
        my_xml_string=common_string+repeated+soo+name[1]+so5+xmin[1]+so6+ymin[1]+so7+xmax[1]+so8+ymax[1]+so9+fine
    myxml = fromstring(my_xml_string)
    tree = ElementTree(myxml)
        
    return tree

In [ ]:
destin="C:/Users/Acanfora/Desktop/motorino_resized/"

path="C:/Users/Acanfora/Desktop/motorino_lungo/"
string_name="DeviceData_#"
string_number=str(181)
estens=".jpg"

lista=os.listdir(path)[10:260]

In [ ]:
#The following cycle resized a set of images taken in lab

for i in range(len(lista)):
    final=destin+str(i)+estens
    image=cv2.imread(path+string_name+str(i+10).zfill(3)+estens)
    height,width,scale=image.shape
    ratio=height/width
    resized_image = cv2.resize(image, (600, int(600*ratio)))
    cv2.imwrite(final,resized_image)

In [ ]:
path = "C:/Users/Acanfora/Desktop/motorino_resized/"
flower_path="C:/Users/Acanfora/Dropbox/magistrale/II_sem/lab/python/fiori/"

#tx, ty = 600, 451 larghezza, altezza immagini generate da mathematica
ex, ey = 290,290 #width, height of modes in my lab pictures
fx,fy=244,244 #flowers dimensions

mylist = os.listdir(flower_path)  #makes the list of flower images
lim=100

classi=['uno','due','tre','quattro','cinque']  #learning classes
#coppie_range=30

In [ ]:
#At the end of this cycle we will have 2500 random images placed in the correct folder and
#in the correct amount as indicated by the guide you can find in the README file.

for i in range(len(lista)):
    for k in range(10):
        if (i%50) < int(0.8*50):
            destin="C:/tensorflow1/models/research/object_detection/images/train/"
        else:
            destin="C:/tensorflow1/models/research/object_detection/images/test/"
        j=(i//50)+1
        if ((i+1)%40==0): 
            print(i,j)
        s=str(j)+"_"+str(k).zfill(2)+"_"+str(i).zfill(3)+".jpg"
        immagine,a1,b1=myrandtransform(path+str(i)+".jpg")
        cv2.imwrite(destin+s,immagine)
        name=classi[j-1]
        tree=createxml('simple',s,[name],destin,str(immagine.shape[1]),str(immagine.shape[0]),[str(a1[0])],[str(a1[1])],[str(b1[0])],[str(b1[1])])
        tree.write(destin+s[:-3]+"xml")
winsound.Beep(500,1000)  #makes a beep when the cycle is done to wake you up after 
#those 2 or 3 minutes it takes

                                                                 